In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%shell

# Both, with custom grid and parallelism
python drive/MyDrive/mad/tests/occlusion_heatmaps_builder.py \
  --heatmap-mode diff \
  --regular /content/drive/MyDrive/mad/code/ViT-SVM/mad_scores/ffhq-cropped/ffhq-cropped/test_using_train_model_with_balanced_data.txt \
  --grid /content/drive/MyDrive/mad/code/ViT-SVM/mad_scores/ffhq-grid-occlusion/ffhq-cropped/test_using_train_model_with_balanced_data.txt \
  --landmark /content/drive/MyDrive/mad/code/ViT-SVM/mad_scores/ffhq-landmark-occlusion/ffhq-cropped/test_using_train_model_with_balanced_data.txt \
  --output output_diff \
  --multiprocessing --workers 12

Streaming output truncated to the last 5000 lines.
[landmark] No match found for '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/ffhq-cropped/morphs-occluded-regions/right_eye/stylegan-interp-W/52017_66002.png' (suffix='stylegan-interp-W/52017_66002.png')
[landmark] No match found for '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/ffhq-cropped/morphs-occluded-regions/right_eye/stylegan-interp-W/52044_42809.png' (suffix='stylegan-interp-W/52044_42809.png')
[landmark] No match found for '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/ffhq-cropped/morphs-occluded-regions/right_eye/stylegan-interp-W/52048_63893.png' (suffix='stylegan-interp-W/52048_63893.png')
[landmark] No match found for '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/ffhq-cropped/morphs-occluded-regions/right_eye/stylegan-interp-W/52052_65184.png' (suffix='stylegan-interp-W/52052_65184.png')
[landmark] No match found for '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/ffhq-cropped/morphs-occluded-regions/righ

In [ ]:
from pathlib import Path
from zipfile import ZipFile, ZIP_DEFLATED
from tqdm import tqdm

# --- Paths ---
src = Path("output_diff")  # folder to zip
zip_local = Path("/content/output_diff.zip")  # temporary zip in Colab local storage
zip_drive = Path("/content/drive/MyDrive/mad/tests/output_diff_heatmaps.zip")  # final zip on Drive

# --- 1. Zip with progress bar ---
files = [f for f in src.rglob("*") if f.is_file()]

print("Zipping files...")
with ZipFile(zip_local, "w", compression=ZIP_DEFLATED) as zipf:
    for f in tqdm(files, desc="Zipping", unit="file"):
        rel = f.relative_to(src)
        zipf.write(f, rel)

# --- 2. Copy zip to Drive with progress bar ---
print("Copying zip to Drive...")
buffer_size = 1024 * 1024  # 1 MB chunks
total_size = zip_local.stat().st_size

with open(zip_local, "rb") as fsrc, open(zip_drive, "wb") as fdst:
    with tqdm(total=total_size, unit="B", unit_scale=True, desc="Copying") as pbar:
        while True:
            buf = fsrc.read(buffer_size)
            if not buf:
                break
            fdst.write(buf)
            pbar.update(len(buf))

print(f"\n✅ Zip saved at: {zip_drive}")

In [ ]:
import time
from google.colab import drive, runtime

drive.flush_and_unmount()

runtime.unassign()